In [1]:
import pandas as pd
import geopandas as gpd
import json
import regex
from shapely.geometry import Point


In [2]:
import pyspark
from pyspark.sql import SparkSession, functions as F

# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("Project 2 Visualisation")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .getOrCreate()
)

22/10/10 03:45:07 WARN Utils: Your hostname, DESKTOP-Q5VPOFC resolves to a loopback address: 127.0.1.1; using 172.31.150.234 instead (on interface eth0)
22/10/10 03:45:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/10 03:45:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/10 03:45:09 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/10/10 03:45:09 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
rental_df = pd.read_csv("../data/curated/proximity_calc_final.csv")

In [4]:
rental_df.head()

,name,prop_type,cost_text,coordinates,add_feat,Beds_no,Baths_no,Parking_no,Acreage / Semi-Rural,Apartment / Unit / Flat,...,school_prox_2_min,school_prox_closest,recre_prox_15km,recre_prox_2_min,recre_prox_closest,shopping_prox_5km,shopping_prox_2_min,shopping_prox_closest,hosp_prox_5km,hosp_closest
0,3309/18 Hoff Boulevard Southbank VIC 3006,Apartment / Unit / Flat,650.0,"[144.9640414, -37.825817]","['built in wardrobes', 'ducted heating']",2,2,0,0,1,...,0,0.76,1010,0,1.25,197,8,0.02,5,3.07
1,5605/70 Southbank Boulevard Southbank VIC 3006,Apartment / Unit / Flat,470.0,"[144.9639881, -37.8231931]","['gas', 'internal laundry', 'intercom', 'floor...",1,1,0,0,1,...,0,1.01,1009,0,1.27,197,8,0.02,5,2.95
2,229/22 Kavanagh Street Southbank VIC 3006,Apartment / Unit / Flat,420.0,"[144.9658569, -37.82284060000001]","['heating', 'balcony deck', 'furnished']",1,1,1,0,1,...,0,0.91,1023,0,1.37,197,8,0.02,5,2.78
3,1112/39 Coventry Street Southbank VIC 3006,Apartment / Unit / Flat,550.0,"[144.9692407, -37.8294033]","['built in wardrobes', 'gas', 'internal laundr...",2,1,1,0,1,...,0,0.85,1000,0,2.00,197,7,0.02,5,3.14
4,1913/39 Coventry Street Southbank VIC 3006,Apartment / Unit / Flat,550.0,"[144.9692407, -37.8294033]","['air conditioning', 'built in wardrobes', 'ga...",2,1,1,0,1,...,0,0.85,1000,0,2.00,197,7,0.02,5,3.14


In [5]:
rental_df['coordinates'] = rental_df['coordinates'].str.replace(' ', '').str.replace('[', '').str.replace(']', '')
rental_df['coordinates'] = rental_df['coordinates'].str.split(',')


/tmp/ipykernel_2759/1507019160.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  rental_df['coordinates'] = rental_df['coordinates'].str.replace(' ', '').str.replace('[', '').str.replace(']', '')


In [6]:
for i in range(0,len(rental_df['coordinates'])):
    rental_df['coordinates'][i] = [float(x) for x in rental_df['coordinates'][i]]

/tmp/ipykernel_2759/4064507194.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rental_df['coordinates'][i] = [float(x) for x in rental_df['coordinates'][i]]


In [7]:
rental_df['coordinates'] = rental_df['coordinates'].apply(Point)

In [8]:
rental_df.head()

,name,prop_type,cost_text,coordinates,add_feat,Beds_no,Baths_no,Parking_no,Acreage / Semi-Rural,Apartment / Unit / Flat,...,school_prox_2_min,school_prox_closest,recre_prox_15km,recre_prox_2_min,recre_prox_closest,shopping_prox_5km,shopping_prox_2_min,shopping_prox_closest,hosp_prox_5km,hosp_closest
0,3309/18 Hoff Boulevard Southbank VIC 3006,Apartment / Unit / Flat,650.0,POINT (144.9640414 -37.825817),"['built in wardrobes', 'ducted heating']",2,2,0,0,1,...,0,0.76,1010,0,1.25,197,8,0.02,5,3.07
1,5605/70 Southbank Boulevard Southbank VIC 3006,Apartment / Unit / Flat,470.0,POINT (144.9639881 -37.8231931),"['gas', 'internal laundry', 'intercom', 'floor...",1,1,0,0,1,...,0,1.01,1009,0,1.27,197,8,0.02,5,2.95
2,229/22 Kavanagh Street Southbank VIC 3006,Apartment / Unit / Flat,420.0,POINT (144.9658569 -37.82284060000001),"['heating', 'balcony deck', 'furnished']",1,1,1,0,1,...,0,0.91,1023,0,1.37,197,8,0.02,5,2.78
3,1112/39 Coventry Street Southbank VIC 3006,Apartment / Unit / Flat,550.0,POINT (144.9692407 -37.8294033),"['built in wardrobes', 'gas', 'internal laundr...",2,1,1,0,1,...,0,0.85,1000,0,2.00,197,7,0.02,5,3.14
4,1913/39 Coventry Street Southbank VIC 3006,Apartment / Unit / Flat,550.0,POINT (144.9692407 -37.8294033),"['air conditioning', 'built in wardrobes', 'ga...",2,1,1,0,1,...,0,0.85,1000,0,2.00,197,7,0.02,5,3.14


In [9]:
#Creates geodataframe from the shapefile with postcodes
postcode_sf = gpd.GeoDataFrame.from_file("../data/raw/rental_zones/POSTCODE_POLYGON.shp")
postcode_sf['geometry'] = postcode_sf['geometry'].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")


In [10]:
postcode_sf = postcode_sf[['POSTCODE','geometry']]
# Creates geodataframe with points of rental properties
rental_points = gpd.GeoDataFrame(rental_df,geometry='coordinates',crs = postcode_sf.crs)
# Merges two geodataframes and checkes if properties are in postcode area
pointInPolys = gpd.tools.sjoin(rental_points, postcode_sf, predicate="within", how='left')

In [11]:
pointInPolys.head()

,name,prop_type,cost_text,coordinates,add_feat,Beds_no,Baths_no,Parking_no,Acreage / Semi-Rural,Apartment / Unit / Flat,...,recre_prox_15km,recre_prox_2_min,recre_prox_closest,shopping_prox_5km,shopping_prox_2_min,shopping_prox_closest,hosp_prox_5km,hosp_closest,index_right,POSTCODE
0,3309/18 Hoff Boulevard Southbank VIC 3006,Apartment / Unit / Flat,650.0,POINT (144.96404 -37.82582),"['built in wardrobes', 'ducted heating']",2,2,0,0,1,...,1010,0,1.25,197,8,0.02,5,3.07,154.0,3006
1,5605/70 Southbank Boulevard Southbank VIC 3006,Apartment / Unit / Flat,470.0,POINT (144.96399 -37.82319),"['gas', 'internal laundry', 'intercom', 'floor...",1,1,0,0,1,...,1009,0,1.27,197,8,0.02,5,2.95,154.0,3006
2,229/22 Kavanagh Street Southbank VIC 3006,Apartment / Unit / Flat,420.0,POINT (144.96586 -37.82284),"['heating', 'balcony deck', 'furnished']",1,1,1,0,1,...,1023,0,1.37,197,8,0.02,5,2.78,154.0,3006
3,1112/39 Coventry Street Southbank VIC 3006,Apartment / Unit / Flat,550.0,POINT (144.96924 -37.82940),"['built in wardrobes', 'gas', 'internal laundr...",2,1,1,0,1,...,1000,0,2.00,197,7,0.02,5,3.14,154.0,3006
4,1913/39 Coventry Street Southbank VIC 3006,Apartment / Unit / Flat,550.0,POINT (144.96924 -37.82940),"['air conditioning', 'built in wardrobes', 'ga...",2,1,1,0,1,...,1000,0,2.00,197,7,0.02,5,3.14,154.0,3006


In [12]:
pointInPolys = pointInPolys.drop('index_right', axis =1)

In [13]:
# Test for locality ( which suburb properties are in)
locality_sf = gpd.read_file("../data/curated/locality/SAL_2021_AUST_GDA94.shp")

In [14]:
locality_sf.head()

,SAL_CODE21,SAL_NAME21,STE_CODE21,STE_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21,SHAPE_Leng,SHAPE_Area,geometry
0,10001,Aarons Pass,1,New South Wales,AUS,Australia,82.7639,http://linked.data.gov.au/dataset/asgsed3/SAL/...,0.554241,0.007975,"POLYGON ((149.82476 -32.84385, 149.83271 -32.8..."
1,10002,Abbotsbury,1,New South Wales,AUS,Australia,4.9788,http://linked.data.gov.au/dataset/asgsed3/SAL/...,0.123051,0.000485,"POLYGON ((150.86522 -33.88265, 150.86479 -33.8..."
2,10003,Abbotsford (NSW),1,New South Wales,AUS,Australia,1.0180,http://linked.data.gov.au/dataset/asgsed3/SAL/...,0.053423,0.000099,"POLYGON ((151.13471 -33.85493, 151.13445 -33.8..."
3,10004,Abercrombie,1,New South Wales,AUS,Australia,2.9775,http://linked.data.gov.au/dataset/asgsed3/SAL/...,0.097338,0.000289,"POLYGON ((149.55191 -33.39281, 149.55148 -33.3..."
4,10005,Abercrombie River,1,New South Wales,AUS,Australia,127.1701,http://linked.data.gov.au/dataset/asgsed3/SAL/...,0.848903,0.012397,"POLYGON ((149.25562 -33.96536, 149.25562 -33.9..."


In [15]:
locality_sf2 = locality_sf.loc[locality_sf['STE_NAME21'] == 'Victoria']
pointInPolys2 = gpd.tools.sjoin(pointInPolys, locality_sf2[['SAL_NAME21','geometry']], predicate="within", how='left')

/tmp/ipykernel_2759/1180955500.py:2: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs + ...
Right CRS: EPSG:4283

  pointInPolys2 = gpd.tools.sjoin(pointInPolys, locality_sf2[['SAL_NAME21','geometry']], predicate="within", how='left')


In [16]:
pointInPolys2.head()

,name,prop_type,cost_text,coordinates,add_feat,Beds_no,Baths_no,Parking_no,Acreage / Semi-Rural,Apartment / Unit / Flat,...,recre_prox_2_min,recre_prox_closest,shopping_prox_5km,shopping_prox_2_min,shopping_prox_closest,hosp_prox_5km,hosp_closest,POSTCODE,index_right,SAL_NAME21
0,3309/18 Hoff Boulevard Southbank VIC 3006,Apartment / Unit / Flat,650.0,POINT (144.96404 -37.82582),"['built in wardrobes', 'ducted heating']",2,2,0,0,1,...,0,1.25,197,8,0.02,5,3.07,3006,6858.0,Southbank
1,5605/70 Southbank Boulevard Southbank VIC 3006,Apartment / Unit / Flat,470.0,POINT (144.96399 -37.82319),"['gas', 'internal laundry', 'intercom', 'floor...",1,1,0,0,1,...,0,1.27,197,8,0.02,5,2.95,3006,6858.0,Southbank
2,229/22 Kavanagh Street Southbank VIC 3006,Apartment / Unit / Flat,420.0,POINT (144.96586 -37.82284),"['heating', 'balcony deck', 'furnished']",1,1,1,0,1,...,0,1.37,197,8,0.02,5,2.78,3006,6858.0,Southbank
3,1112/39 Coventry Street Southbank VIC 3006,Apartment / Unit / Flat,550.0,POINT (144.96924 -37.82940),"['built in wardrobes', 'gas', 'internal laundr...",2,1,1,0,1,...,0,2.00,197,7,0.02,5,3.14,3006,6858.0,Southbank
4,1913/39 Coventry Street Southbank VIC 3006,Apartment / Unit / Flat,550.0,POINT (144.96924 -37.82940),"['air conditioning', 'built in wardrobes', 'ga...",2,1,1,0,1,...,0,2.00,197,7,0.02,5,3.14,3006,6858.0,Southbank


In [17]:
pointInPolys2.columns

Index(['name', 'prop_type', 'cost_text', 'coordinates', 'add_feat', 'Beds_no',
       'Baths_no', 'Parking_no', 'Acreage / Semi-Rural',
       'Apartment / Unit / Flat', 'Block of Units', 'Carspace', 'Duplex',
       'House', 'New Apartments / Off the Plan', 'New House & Land',
       'Penthouse', 'Rural', 'Semi-Detached', 'Studio', 'Terrace', 'Townhouse',
       'Villa', 'air conditioning', 'balcony  deck', 'bath',
       'built in wardrobes', 'close to schools', 'close to shops',
       'close to transport', 'dishwasher', 'ducted heating', 'ensuite',
       'floorboards', 'fully fenced', 'furnished', 'garden  courtyard', 'gas',
       'heating', 'intercom', 'internal laundry', 'no extra features listed',
       'pets allowed', 'remote garage', 'secure parking', 'shed',
       'split system heating', 'study', 'cbd_dur', 'cbd_dist',
       'train_prox_2km', 'train_prox_2_min', 'train_prox_closest',
       'school_prox_15km', 'school_prox_2_min', 'school_prox_closest',
       'recre_pro

In [18]:
pointInPolys2.head()

,name,prop_type,cost_text,coordinates,add_feat,Beds_no,Baths_no,Parking_no,Acreage / Semi-Rural,Apartment / Unit / Flat,...,recre_prox_2_min,recre_prox_closest,shopping_prox_5km,shopping_prox_2_min,shopping_prox_closest,hosp_prox_5km,hosp_closest,POSTCODE,index_right,SAL_NAME21
0,3309/18 Hoff Boulevard Southbank VIC 3006,Apartment / Unit / Flat,650.0,POINT (144.96404 -37.82582),"['built in wardrobes', 'ducted heating']",2,2,0,0,1,...,0,1.25,197,8,0.02,5,3.07,3006,6858.0,Southbank
1,5605/70 Southbank Boulevard Southbank VIC 3006,Apartment / Unit / Flat,470.0,POINT (144.96399 -37.82319),"['gas', 'internal laundry', 'intercom', 'floor...",1,1,0,0,1,...,0,1.27,197,8,0.02,5,2.95,3006,6858.0,Southbank
2,229/22 Kavanagh Street Southbank VIC 3006,Apartment / Unit / Flat,420.0,POINT (144.96586 -37.82284),"['heating', 'balcony deck', 'furnished']",1,1,1,0,1,...,0,1.37,197,8,0.02,5,2.78,3006,6858.0,Southbank
3,1112/39 Coventry Street Southbank VIC 3006,Apartment / Unit / Flat,550.0,POINT (144.96924 -37.82940),"['built in wardrobes', 'gas', 'internal laundr...",2,1,1,0,1,...,0,2.00,197,7,0.02,5,3.14,3006,6858.0,Southbank
4,1913/39 Coventry Street Southbank VIC 3006,Apartment / Unit / Flat,550.0,POINT (144.96924 -37.82940),"['air conditioning', 'built in wardrobes', 'ga...",2,1,1,0,1,...,0,2.00,197,7,0.02,5,3.14,3006,6858.0,Southbank


In [19]:
pointInPolys2 = pointInPolys2 \
    .merge(locality_sf2[['SAL_NAME21', 'geometry']], left_on='SAL_NAME21', right_on='SAL_NAME21')
pointInPolys2.head()

,name,prop_type,cost_text,coordinates,add_feat,Beds_no,Baths_no,Parking_no,Acreage / Semi-Rural,Apartment / Unit / Flat,...,recre_prox_closest,shopping_prox_5km,shopping_prox_2_min,shopping_prox_closest,hosp_prox_5km,hosp_closest,POSTCODE,index_right,SAL_NAME21,geometry
0,3309/18 Hoff Boulevard Southbank VIC 3006,Apartment / Unit / Flat,650.0,POINT (144.96404 -37.82582),"['built in wardrobes', 'ducted heating']",2,2,0,0,1,...,1.25,197,8,0.02,5,3.07,3006,6858.0,Southbank,"POLYGON ((144.96138 -37.82051, 144.96303 -37.8..."
1,5605/70 Southbank Boulevard Southbank VIC 3006,Apartment / Unit / Flat,470.0,POINT (144.96399 -37.82319),"['gas', 'internal laundry', 'intercom', 'floor...",1,1,0,0,1,...,1.27,197,8,0.02,5,2.95,3006,6858.0,Southbank,"POLYGON ((144.96138 -37.82051, 144.96303 -37.8..."
2,229/22 Kavanagh Street Southbank VIC 3006,Apartment / Unit / Flat,420.0,POINT (144.96586 -37.82284),"['heating', 'balcony deck', 'furnished']",1,1,1,0,1,...,1.37,197,8,0.02,5,2.78,3006,6858.0,Southbank,"POLYGON ((144.96138 -37.82051, 144.96303 -37.8..."
3,1112/39 Coventry Street Southbank VIC 3006,Apartment / Unit / Flat,550.0,POINT (144.96924 -37.82940),"['built in wardrobes', 'gas', 'internal laundr...",2,1,1,0,1,...,2.00,197,7,0.02,5,3.14,3006,6858.0,Southbank,"POLYGON ((144.96138 -37.82051, 144.96303 -37.8..."
4,1913/39 Coventry Street Southbank VIC 3006,Apartment / Unit / Flat,550.0,POINT (144.96924 -37.82940),"['air conditioning', 'built in wardrobes', 'ga...",2,1,1,0,1,...,2.00,197,7,0.02,5,3.14,3006,6858.0,Southbank,"POLYGON ((144.96138 -37.82051, 144.96303 -37.8..."


In [20]:
property_scraped_suburb = pointInPolys2.groupby('SAL_NAME21')['cost_text'].count()
pointInPolys2 = pointInPolys2.drop('index_right', axis = 1)
mean_rent_suburb = pd.DataFrame(pointInPolys2.groupby(['SAL_NAME21','POSTCODE'])['cost_text'].median()).reset_index()

In [21]:
mean_rent_suburb['no_prop_scraped'] = 0


In [22]:
pointInPolys2.columns

Index(['name', 'prop_type', 'cost_text', 'coordinates', 'add_feat', 'Beds_no',
       'Baths_no', 'Parking_no', 'Acreage / Semi-Rural',
       'Apartment / Unit / Flat', 'Block of Units', 'Carspace', 'Duplex',
       'House', 'New Apartments / Off the Plan', 'New House & Land',
       'Penthouse', 'Rural', 'Semi-Detached', 'Studio', 'Terrace', 'Townhouse',
       'Villa', 'air conditioning', 'balcony  deck', 'bath',
       'built in wardrobes', 'close to schools', 'close to shops',
       'close to transport', 'dishwasher', 'ducted heating', 'ensuite',
       'floorboards', 'fully fenced', 'furnished', 'garden  courtyard', 'gas',
       'heating', 'intercom', 'internal laundry', 'no extra features listed',
       'pets allowed', 'remote garage', 'secure parking', 'shed',
       'split system heating', 'study', 'cbd_dur', 'cbd_dist',
       'train_prox_2km', 'train_prox_2_min', 'train_prox_closest',
       'school_prox_15km', 'school_prox_2_min', 'school_prox_closest',
       'recre_pro

In [23]:
# Removes error message:
pd.options.mode.chained_assignment = None  

In [24]:
for i in range(0,len(mean_rent_suburb)):
    mean_rent_suburb['no_prop_scraped'][i] = property_scraped_suburb[mean_rent_suburb['SAL_NAME21'][i]]

In [25]:
# Calculates the median number of each feature found by properties in each suburb and postcode
median_train_prox_2km = pd.DataFrame(pointInPolys2.groupby(['SAL_NAME21','POSTCODE'])['train_prox_2km'].median()).reset_index()
median_school_prox_2_min =pd.DataFrame(pointInPolys2.groupby(['SAL_NAME21','POSTCODE'])['school_prox_2_min'].median()).reset_index()
median_recre_prox_2_min = pd.DataFrame(pointInPolys2.groupby(['SAL_NAME21','POSTCODE'])['recre_prox_2_min'].median()).reset_index()
median_shopping_prox_2_min = pd.DataFrame(pointInPolys2.groupby(['SAL_NAME21','POSTCODE'])['shopping_prox_2_min'].median()).reset_index()
median_hosp_closest = pd.DataFrame(pointInPolys2.groupby(['SAL_NAME21','POSTCODE'])['hosp_closest'].median()).reset_index()

In [26]:
# Joins data together
suburb_prox_df = mean_rent_suburb.join(median_train_prox_2km['train_prox_2km'])
suburb_prox_df = suburb_prox_df.join(median_school_prox_2_min['school_prox_2_min'])
suburb_prox_df = suburb_prox_df.join(median_recre_prox_2_min['recre_prox_2_min'])
suburb_prox_df = suburb_prox_df.join(median_shopping_prox_2_min['shopping_prox_2_min'])
suburb_prox_df = suburb_prox_df.join(median_hosp_closest['hosp_closest'])

In [27]:
suburb_prox_df.head()

,SAL_NAME21,POSTCODE,cost_text,no_prop_scraped,train_prox_2km,school_prox_2_min,recre_prox_2_min,shopping_prox_2_min,hosp_closest
0,Abbotsford (Vic.),3067,475.0,47,2.0,1.0,1.0,7.0,2.150
1,Aberfeldie,3040,480.0,12,0.0,1.5,2.5,7.0,7.345
2,Aintree,3336,445.0,12,0.0,0.0,0.0,7.0,13.540
3,Airport West,3042,440.0,16,0.0,0.0,0.0,7.0,7.895
4,Albanvale,3021,350.0,5,0.0,0.0,1.0,7.0,6.020


In [28]:
income_by_postcode = pd.read_excel("../data/raw/income_suburb.xlsx", 'Table 8')

In [29]:
# Change the headers
income_by_postcode.columns = income_by_postcode.iloc[0]
income_by_postcode = income_by_postcode.drop([income_by_postcode.index[0]])


In [30]:
income_by_postcode.head()

,State/\nTerritory,Postcode2,Individuals 2003–04 \nno.,Median3 taxable income 2003–04 \n$,Average3 taxable income 2003–04 \n$,Individuals 2013–14 \nno.,Median3 taxable income 2013–14 \n$,Average3 taxable income 2013–14 \n$,Individuals 2014–15 \nno.,Median3 taxable income 2014–15 \n$,...,Average3 taxable income 2016–17\n$,Individuals 2017–18\nno.,Median3 taxable income 2017–18\n$,Average3 taxable income 2017–18\n$,Individuals 2018–19\nno.,Median3 taxable income 2018–19\n$,Average3 taxable income 2018–19\n$,Individuals 2019–20\nno.,Median3 taxable income 2019–20\n$,Average3 taxable income 2019–20\n$
1,ACT,2600,5692,42327,55819,5712,66815,94265,5592,68327,...,102131,5962,73941,107634,5953,76177,112280,5945,78636,119465
2,ACT,2601,1833,35584,47294,2777,46880,66876,2792,48061,...,64948,2810,42957,64939,3083,39781,64871,3159,45914,67940
3,ACT,2602,17431,35997,41004,19072,57584,68424,19096,57245,...,71274,20613,59921,72821,21263,62841,75693,22009,64315,79393
4,ACT,2603,5973,45229,62762,6210,70655,116150,6305,72177,...,110394,6850,75869,115723,6978,79369,119441,7165,81877,126534
5,ACT,2604,6053,41732,49505,7205,67865,81898,7464,69440,...,87389,8265,74523,90095,8478,76609,91603,8617,77737,95374


In [31]:
# Take the latest median income to calculate affordability
latest_median_income = pd.DataFrame(income_by_postcode, columns=['State/\nTerritory','Postcode2','Median3 taxable income 2019–20\n$'])
# Filter data for Victoria Only
latest_median_income = latest_median_income[latest_median_income['State/\nTerritory'] == 'VIC']
latest_median_income = latest_median_income.drop('State/\nTerritory', axis = 1)
latest_median_income.rename(columns={'Median3 taxable income 2019–20\n$':'Median Taxable Income', 'Postcode2':'POSTCODE'}, inplace=True)

In [32]:
latest_median_income.columns

Index(['POSTCODE', 'Median Taxable Income'], dtype='object')

In [33]:
latest_median_income.reset_index(drop= True, inplace = True)

suburb_prox_df = suburb_prox_df.astype({'POSTCODE':'int'})

In [34]:
afford_df = suburb_prox_df
afford_df.head()

,SAL_NAME21,POSTCODE,cost_text,no_prop_scraped,train_prox_2km,school_prox_2_min,recre_prox_2_min,shopping_prox_2_min,hosp_closest
0,Abbotsford (Vic.),3067,475.0,47,2.0,1.0,1.0,7.0,2.150
1,Aberfeldie,3040,480.0,12,0.0,1.5,2.5,7.0,7.345
2,Aintree,3336,445.0,12,0.0,0.0,0.0,7.0,13.540
3,Airport West,3042,440.0,16,0.0,0.0,0.0,7.0,7.895
4,Albanvale,3021,350.0,5,0.0,0.0,1.0,7.0,6.020


In [35]:
afford_df['median income'] = 0

In [36]:
for i in range(0,len(afford_df['POSTCODE'])):
    if len(latest_median_income[latest_median_income['POSTCODE'] == afford_df['POSTCODE'][i]]) != 0:
        afford_df['median income'][i] = latest_median_income[latest_median_income['POSTCODE'] == afford_df['POSTCODE'][i]]['Median Taxable Income'].values[0]
    else:
        afford_df['median income'][i] = -1


In [37]:
# Some of the suburbs which were scraped were not listed in the median income by suburb dataset
# From a quick google search, most of these are suburbs will little to no population so can be removed without affecting analysis
afford_df[afford_df['median income'] == -1]

,SAL_NAME21,POSTCODE,cost_text,no_prop_scraped,train_prox_2km,school_prox_2_min,recre_prox_2_min,shopping_prox_2_min,hosp_closest,median income
49,Barunah Park,3329,420.0,1,0.0,0.0,0.0,7.0,34.31,-1
191,Cressy (Vic.),3322,370.0,2,0.0,0.0,0.0,7.0,54.08,-1
323,Hexham (Vic.),3273,300.0,1,0.0,0.0,0.0,7.0,42.89,-1
456,Monegeetta,3433,520.0,1,0.0,0.0,0.0,7.0,52.43,-1
535,Peterborough (Vic.),3270,650.0,1,0.0,0.0,0.0,7.0,22.57,-1
593,Shelford,3329,380.0,1,0.0,0.0,0.0,7.0,31.28,-1
601,Somerton (Vic.),3062,320.0,1,0.0,0.0,0.0,7.0,9.76,-1
660,Torrumbarry,3562,295.0,1,0.0,0.0,0.0,7.0,63.41,-1
721,Yan Yean,3755,650.0,1,0.0,0.0,1.0,7.0,25.54,-1


In [38]:
afford_df = afford_df[afford_df['median income'] != -1]
afford_df['affordability'] = afford_df['cost_text']*52/afford_df['median income']
ranked_afford = afford_df.sort_values(['affordability','no_prop_scraped'], ascending = [True, False])

In [39]:
# Checking distribution of how many properties were scraped for their respective suburbs
afford_df['no_prop_scraped'].value_counts()

1      144
2       75
3       29
6       28
4       26
      ... 
137      1
77       1
71       1
73       1
68       1
Name: no_prop_scraped, Length: 91, dtype: int64

In [40]:
ranked_afford.head()

,SAL_NAME21,POSTCODE,cost_text,no_prop_scraped,train_prox_2km,school_prox_2_min,recre_prox_2_min,shopping_prox_2_min,hosp_closest,median income,affordability
294,Golden Point (Mount Alexander - Vic.),3451,2.0,1,0.0,0.0,0.0,7.0,9.40,42632,0.002439
183,Craigie (Vic.),3465,2.0,1,0.0,0.0,0.0,7.0,45.03,38546,0.002698
550,Rainbow,3424,100.0,1,0.0,1.0,9.0,7.0,68.53,45269,0.114869
413,Macleod (Vic.),3085,225.0,41,0.0,0.0,0.0,7.0,5.25,54252,0.215660
105,Bundalong,3730,175.0,1,0.0,0.0,0.0,7.0,95.13,40983,0.222043


# Livability

In [41]:
# Remove extreme values, these are mostly results of misinput
afford_df = afford_df[~(afford_df['cost_text'] <= 100)]  
afford_df = afford_df[~(afford_df['cost_text'] >= 3000)]  
afford_df = afford_df[~(afford_df['no_prop_scraped'] <= 5)]  
afford_df = afford_df[~(afford_df['affordability'] >= 5)]  
afford_df = afford_df[~(afford_df['affordability'] <= 0.1)]  


In [42]:
afford_df.columns

Index(['SAL_NAME21', 'POSTCODE', 'cost_text', 'no_prop_scraped',
       'train_prox_2km', 'school_prox_2_min', 'recre_prox_2_min',
       'shopping_prox_2_min', 'hosp_closest', 'median income',
       'affordability'],
      dtype='object')

In [43]:
inc_score_label = [1,2,3,4,5]
dec_score_label = [5,4,3,2,1]

In [44]:
# There are only two options so do not include this for scoring as it will not actually contribute in a meaningful way
afford_df['shopping_prox_2_min'].value_counts()

7.0    420
8.0      1
Name: shopping_prox_2_min, dtype: int64

In [45]:
# Bins the distribution of each of the features and gives them a score out of 5
train_score = pd.cut(afford_df['train_prox_2km'],5, labels =inc_score_label)
school_score = pd.cut(afford_df['school_prox_2_min'], 5, labels = inc_score_label)
recre_score = pd.cut(afford_df['recre_prox_2_min'], 5, labels = inc_score_label)
hosp_score= pd.cut(afford_df['hosp_closest'], 5, labels = [5,4,3,2,1])
# Ideally we want the top scoring suburbs to have less than 0.3 affordability
afford_score= pd.cut(afford_df['affordability'], bins = [0.2,0.3,0.35,0.5,0.65,2], labels = dec_score_label)


In [46]:
# Entering scores into dataframe
afford_df['train_score'] = train_score
afford_df['school_score'] = school_score
afford_df['recre_score'] = recre_score
afford_df['hosp_score'] = hosp_score
afford_df['afford_score'] = afford_score
# Fix datatypes
afford_df['train_score'].astype("int")
afford_df['school_score'].astype("int")
afford_df['recre_score'].astype("int")
afford_df['hosp_score'].astype("int")
afford_df['afford_score'].astype("int")

0      3
1      3
2      3
3      3
5      1
      ..
714    2
715    3
718    3
725    3
726    2
Name: afford_score, Length: 421, dtype: int64

In [47]:
afford_df.head()

,SAL_NAME21,POSTCODE,cost_text,no_prop_scraped,train_prox_2km,school_prox_2_min,recre_prox_2_min,shopping_prox_2_min,hosp_closest,median income,affordability,train_score,school_score,recre_score,hosp_score,afford_score
0,Abbotsford (Vic.),3067,475.0,47,2.0,1.0,1.0,7.0,2.150,62456,0.395478,2,2,1,5,3
1,Aberfeldie,3040,480.0,12,0.0,1.5,2.5,7.0,7.345,58719,0.425075,1,2,1,5,3
2,Aintree,3336,445.0,12,0.0,0.0,0.0,7.0,13.540,53320,0.433983,1,1,1,5,3
3,Airport West,3042,440.0,16,0.0,0.0,0.0,7.0,7.895,55315,0.413631,1,1,1,5,3
5,Albert Park (Vic.),3206,925.0,22,0.0,2.0,2.0,7.0,6.050,71843,0.669515,1,3,1,5,1


In [48]:
# Adds up all the features to find the overall livability score, total score of: 5*5 =25
afford_df['Total_Score']= afford_df.iloc[:, -5:].sum(axis=1)

In [49]:
afford_df.head()

,SAL_NAME21,POSTCODE,cost_text,no_prop_scraped,train_prox_2km,school_prox_2_min,recre_prox_2_min,shopping_prox_2_min,hosp_closest,median income,affordability,train_score,school_score,recre_score,hosp_score,afford_score,Total_Score
0,Abbotsford (Vic.),3067,475.0,47,2.0,1.0,1.0,7.0,2.150,62456,0.395478,2,2,1,5,3,13
1,Aberfeldie,3040,480.0,12,0.0,1.5,2.5,7.0,7.345,58719,0.425075,1,2,1,5,3,12
2,Aintree,3336,445.0,12,0.0,0.0,0.0,7.0,13.540,53320,0.433983,1,1,1,5,3,11
3,Airport West,3042,440.0,16,0.0,0.0,0.0,7.0,7.895,55315,0.413631,1,1,1,5,3,11
5,Albert Park (Vic.),3206,925.0,22,0.0,2.0,2.0,7.0,6.050,71843,0.669515,1,3,1,5,1,11


In [50]:
# Sorts the suburbs from highest overall score to lowest: finding the most Livable suburbs by these metrics
ranked_suburbs = afford_df.sort_values('Total_Score', ascending = False)

In [51]:
ranked_suburbs.head(15)

,SAL_NAME21,POSTCODE,cost_text,no_prop_scraped,train_prox_2km,school_prox_2_min,recre_prox_2_min,shopping_prox_2_min,hosp_closest,median income,affordability,train_score,school_score,recre_score,hosp_score,afford_score,Total_Score
311,Hawthorn (Vic.),3122,450.0,143,3.0,4.0,7.0,7.0,4.250,61024,0.383456,3,5,3,5,3,19
312,Hawthorn East,3123,490.0,91,3.0,3.0,9.0,7.0,5.420,61518,0.414188,3,4,4,5,3,19
286,Glen Iris (Vic.),3123,430.0,81,2.0,4.0,6.0,7.0,4.830,61518,0.363471,2,5,3,5,3,18
264,Fitzroy North,3068,550.0,46,3.0,2.0,8.0,7.0,3.355,63406,0.451061,3,3,4,5,3,18
609,South Yarra,3141,530.0,211,3.0,2.0,8.0,7.0,3.340,65707,0.419438,3,3,4,5,3,18
545,Prahran,3181,450.0,91,3.0,2.0,6.0,7.0,4.290,64796,0.361133,3,3,3,5,3,17
265,Flemington,3031,467.5,32,5.0,2.0,0.0,7.0,3.600,59663,0.407455,5,3,1,5,3,17
663,Travancore,3032,405.0,6,3.0,3.0,3.0,7.0,2.295,54220,0.388418,3,4,2,5,3,17
41,Ballarat North,3350,300.0,6,0.0,1.5,12.5,7.0,17.295,47466,0.328656,1,2,5,5,4,17
36,Balaclava (Vic.),3183,537.5,38,3.0,3.5,3.0,7.0,3.295,56561,0.494157,3,4,2,5,3,17


In [52]:
ranked_suburbs.to_csv("../data/curated/livability_suburbs_ranked")